# DG Schemes [(Notebook)][1]

[1]: _downloads/laplace-dg.ipynb
show the
- dgscheme
- galerkin schemes

In [ ]:
import numpy as np
import scipy.sparse as sparse


def is_symmetric(m):
    """Check if a sparse matrix is symmetric

    Parameters
    ----------
    m : array or sparse matrix
        A square matrix.

    Returns
    -------
    check : bool
        The check result.

    """
    if m.shape[0] != m.shape[1]:
        raise ValueError('m must be a square matrix')

    if not isinstance(m, sparse.coo_matrix):
        m = sparse.coo_matrix(m)

    r, c, v = m.row, m.col, m.data
    tril_no_diag = r > c
    triu_no_diag = c > r

    if triu_no_diag.sum() != tril_no_diag.sum():
        return False

    rl = r[tril_no_diag]
    cl = c[tril_no_diag]
    vl = v[tril_no_diag]
    ru = r[triu_no_diag]
    cu = c[triu_no_diag]
    vu = v[triu_no_diag]

    sortl = np.lexsort((cl, rl))
    sortu = np.lexsort((ru, cu))
    vl = vl[sortl]
    vu = vu[sortu]

    check = np.allclose(vl, vu)

    return check

Let us consider a simple Laplace problem with Dirichlet boundary conditions:
\begin{equation*}
  \begin{aligned}
    -\Delta u &= \sin(\pi x_1) \sin(\pi x_2) && \text{in $\Omega$}, \\
            u &= 0 && \text{on $\partial\Omega$}.
  \end{aligned}
\end{equation*}

First, we need to set up a computational grid and a discontinuous ansatz space on it. Here, we use the orthonormal discontinuous space:

In [ ]:
try:
    %matplotlib inline # can also use notebook or nbagg
except:
    pass
from dune.grid import cartesianDomain
from dune.fem import parameter
from dune.fem.plotting import plotPointData as plot

import dune.create as create

parameter.append({"fem.verboserank": 0, "istl.preconditioning.method": "amg", "istl.preconditioning.iterations": 1, "istl.preconditioning.relaxation": 1.2})

grid = create.grid("ALUCube", cartesianDomain([0,0],[1,1],[16,16]), dimgrid=2)
spc = create.space("dgonb", grid, dimrange=1, order=2, storage="eigen")

The classical IPDG method for this problem reads
\begin{equation*}
  \int_\Omega \nabla u\,\nabla \varphi\,dx
    - \int_\Gamma ([[u]] \otimes \vec{n} : \{\{\nabla \varphi\}\} + \{\{\nabla u\}\} : [[\varphi]] \otimes \vec{n}\,dx
    + \int_\Gamma \frac{\mu}{h} [[u]] [[\varphi]]
    = 0.
\end{equation*}

The following code implements this equation in UFL notation:

In [ ]:
import math
import ufl
from ufl import inner, outer, jump, avg, grad, dx, ds, dS

from dune.ufl import Space

uflSpace = Space((grid.dimGrid, grid.dimWorld), 1)
u = ufl.TrialFunction(uflSpace)
v = ufl.TestFunction(uflSpace)
x = ufl.SpatialCoordinate(uflSpace.cell())
n, h = ufl.FacetNormal(uflSpace.cell()), ufl.MinFacetEdgeLength(uflSpace.cell())
mu = 10. / (h("+")+h("-"))

# use a simple Neuman problem first for testing
base = (inner(grad(u), grad(v)) + inner(u,v) ) * dx
a = base - ( inner(outer(jump(u), jump(n)), avg(grad(v))) 
         -   inner(avg(grad(u)), outer(jump(v), jump(n))) ) * 0.5 * dS
a += mu * inner(jump(u), jump(v)) * dS
# a += (inner(outer(u, n), grad(v)) - inner(grad(u), outer(v, n))) * ds
# a += mu * inner(u, v) * ds

# lets provide an exact solution and compute the forcing from that
exact = [ufl.cos(ufl.pi*x[0])*ufl.cos(ufl.pi*x[1])]
b = ufl.replace(base, {u: ufl.as_vector(exact)} )

Next, we compile this into the *integrands*, plug them into the *galerkin* scheme and solve the problem:

In [ ]:
newtonParameter = {"linabstol": 1e-13, "linreduction": 1e-5, "tolerance": 1e-3, "verbose": "true", "linear.verbose": "true"}

model = create.model("integrands", grid, a == b)
scheme = create.scheme("galerkin", model, spc, solver="bicgstab",
                      parameters={"fem.solver.newton." + k: v for k, v in newtonParameter.items()})
dgmodel = create.model("integrands", grid, base == b)
dgscheme = create.scheme("dg", dgmodel, spc, solver="cg",
                       parameters={"fem.solver.newton." + k: v for k, v in newtonParameter.items()})

uh, _ = scheme.solve()
dguh, _ = dgscheme.solve()


The result looks as follows:

In [ ]:
plot(uh,level=3)
plot(dguh,level=3)

In [ ]:
matrix = scheme.assemble(uh)
dgmatrix = dgscheme.assemble(uh)
print("integrals: ", is_symmetric(matrix))
print("dgelliptic: ",is_symmetric(dgmatrix))

In [ ]:
# grid.hierarchicalGrid.globalRefine(2)
# uh, _ = scheme.solve()
# plot(uh,level=3)